In [1]:
import hvplot.pandas
import holoviews as hv, pandas as pd, colorcet as cc, numpy as np
import panel as pn
from holoviews.element import Points
from holoviews.element.tiles import EsriImagery
from holoviews.operation.datashader import datashade
hv.extension('bokeh')
pn.extension('bokeh')

In [2]:
def latlong_to_mercator(lat, lon):
    """Convert latitude and longitude to Web Mercator coordinates."""
    lat_rad = np.radians(lat)
    lon_rad = np.radians(lon)
    x = 6378137 * lon_rad
    y = 6378137 * np.log(np.tan(np.pi / 4 + lat_rad / 2))
    return x, y

In [3]:
df = pd.read_csv('crashes.csv', usecols = ['LATITUDE', 'LONGITUDE', 'CRASH TIME', 'CRASH DATE'])   #read the csv file
df = df.dropna()  #remove rows with missing values in latitude and longitude columns
df['CRASH_TS'] = pd.to_datetime(df['CRASH TIME'], format='%H:%M', errors='coerce') #Parse crash time into a datatime to sort between hours
df['CRASH_DT'] = pd.to_datetime(df['CRASH DATE'], format='%m/%d/%Y', errors='coerce') #Parse crash date into a datetime to sort by date
df = df.dropna(subset=['CRASH_TS']) #Drop rows that did not get parsed
df = df.dropna(subset=['CRASH_DT']) #Drop rows that did not get parsed
df = df[(df['LATITUDE'] != 0) & (df['LONGITUDE'] != 0)]  #remove rows with zero values in latitude and longitude columns
df = df[df['LATITUDE'].between(40, 42)]
df = df[df['LONGITUDE'].between(-75, -72)]
df['x'], df['y'] = latlong_to_mercator(df['LATITUDE'], df['LONGITUDE'])  #convert latitude and longitude to Web Mercator coordinates
df['hour'] = df['CRASH_TS'].dt.hour #Make a new row that only has the hour
df.sort_values(by='CRASH_DT', inplace=True)#sort df by date
df.to_csv('updated_crashes.csv', index=False)  #save the new df to a new csv file

In [4]:
len(df)

1926916

In [5]:
df.head(20)

,CRASH DATE,CRASH TIME,LATITUDE,LONGITUDE,CRASH_TS,CRASH_DT,x,y,hour
2100961,07/01/2012,17:30,40.731982,-73.981966,1900-01-01 17:30:00,2012-07-01,-8.235635e+06,4.972889e+06,17
2103776,07/01/2012,16:55,40.658030,-73.947433,1900-01-01 16:55:00,2012-07-01,-8.231791e+06,4.962031e+06,16
2102849,07/01/2012,19:55,40.689218,-73.917649,1900-01-01 19:55:00,2012-07-01,-8.228475e+06,4.966609e+06,19
2102844,07/01/2012,16:00,40.615643,-74.067283,1900-01-01 16:00:00,2012-07-01,-8.245132e+06,4.955813e+06,16
2099828,07/01/2012,16:30,40.691417,-73.757941,1900-01-01 16:30:00,2012-07-01,-8.210696e+06,4.966932e+06,16
2102838,07/01/2012,12:40,40.612834,-74.033648,1900-01-01 12:40:00,2012-07-01,-8.241388e+06,4.955401e+06,12
2099817,07/01/2012,21:55,40.767889,-73.981512,1900-01-01 21:55:00,2012-07-01,-8.235584e+06,4.978165e+06,21
2099815,07/01/2012,11:45,40.576827,-73.965096,1900-01-01 11:45:00,2012-07-01,-8.233757e+06,4.950123e+06,11
2102829,07/01/2012,9:00,40.744131,-73.927047,1900-01-01 09:00:00,2012-07-01,-8.229521e+06,4.974674e+06,9
2102818,07/01/2012,20:15,40.670447,-73.981925,1900-01-01 20:15:00,2012-07-01,-8.235630e+06,4.963854e+06,20


In [6]:
# %%time
# map_tiles = EsriImagery().opts(alpha=0.5, width=900, height =480, bgcolor='black')
# points = hv.Points(df,  ['x', 'y'])
# crashes = datashade(points, cmap=cc.fire, width=900, height=480)
# map_tiles * crashes

In [7]:
# plot = df.hvplot(
#     'x',
#     'y',
#     kind = 'scatter',
#     rasterize = True,
#     cmap = cc.fire,
#     cnorm = 'eq_hist'
# )
# map_tiles * plot

In [8]:
start_picker = pn.widgets.DatePicker(
    name="Start Date",
    value=df.CRASH_DT.min().date(),
    start=df.CRASH_DT.min().date(),
    end=df.CRASH_DT.max().date(),
)
end_picker = pn.widgets.DatePicker(
    name='End Date',
    value=df.CRASH_DT.max().date(),
    start=df.CRASH_DT.min().date(),
    end=df.CRASH_DT.max().date(),
)

In [9]:
def filtered_map(start,end):
    # slice by date
    mask = (df.CRASH_DT >= pd.Timestamp(start)) & (df.CRASH_DT <= pd.Timestamp(end))
    sub = df[mask]

    # rebuild static-shader layers for each hour
    hourly_layers = {
        h: datashade(
               Points(sub[sub.hour == h], ['x','y']),
               cmap=cc.fire, width=900, height=680,
               dynamic=False    # returns a plain RGB element
           )
        for h in sorted(sub.hour.unique())
    }

    # %% Package into a slider & overlay on EsriImagery
    hmap = hv.HoloMap(hourly_layers, kdims='Hour of Day').opts(
        legend_position='bottom',                               # place slider on left
        scalebar_opts={'orientation': 'horizontal', 'width': 80}, # make it vertical & narrow
        xlabel=None, ylabel=None,                             # hide axis labels
        bgcolor='black',                                      # map background
        toolbar='below',
    )

    tiles = EsriImagery().opts(alpha=0.5, width=900, height=680)
    return (tiles * hmap).opts(
        title="NYC Crashes by Hour",
        frame_width=900, frame_height=480
    )

In [10]:
map_view = pn.bind(filtered_map, start=start_picker, end=end_picker)

dashboard = pn.Column(
    pn.Row(start_picker, end_picker),
    map_view
)

dashboard.servable()

Column
    [0] Row
        [0] DatePicker(end=datetime.date(2025, ..., name='Start Date', start=datetime.date(2012, 7, 1), value=datetime.date(2012, 7, 1))
        [1] DatePicker(end=datetime.date(2025, ..., name='End Date', start=datetime.date(2012, 7, 1), value=datetime.date(2025, ...)
    [1] ParamFunction(function, _pane=Row, defer_load=False)

In [11]:
# # %% Build static‐shaded layers for each hour
# hourly_layers = {
#     h: datashade(
#            Points(df[df.hour == h], ['x','y']),
#            cmap=cc.fire, width=900, height=680,
#            dynamic=False    # returns a plain RGB element
#        )
#     for h in sorted(df.hour.unique())
# }

# # %% Package into a slider & overlay on EsriImagery
# hmap = hv.HoloMap(hourly_layers, kdims='Hour of Day').opts(
#     legend_position='bottom',                               # place slider on left
#     scalebar_opts={'orientation': 'horizontal', 'width': 80}, # make it vertical & narrow
#     title="NYC Crashes by Hour",
#     xlabel=None, ylabel=None,                             # hide axis labels
#     bgcolor='black'                                       # map background
# )

# tiles = EsriImagery().opts(alpha=0.5, width=900, height=680)
# (tiles * hmap).opts(frame_width=900, frame_height=480, text_color="red")